In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

- cnn: https://pyimagesearch.com/2021/07/19/pytorch-training-your-first-convolutional-neural-network-cnn/
- differentSize: https://learnopencv.com/fully-convolutional-image-classification-on-arbitrary-sized-image/
- spp implementatiojn: https://github.com/revidee/pytorch-pyramid-pooling;;;; https://github.com/yifanjiang19/sppnet-pytorch/tree/master

In [1]:
import numpy as np
import pandas as pd
import pickle
import os

## for visualisation
import matplotlib.pyplot as plt

## for dataset
import torch
from torch.utils.data import Dataset, DataLoader

## extraction of data from files 

In [2]:
## extracting the annotations
csvPath = '/kaggle/input/d/sabyasachi96/profiling-of-spheroids/AllAnnotatedCSVDic.pickle'
with open(csvPath, 'rb') as f:
    data = pickle.load(f)
    
complete_df = pd.DataFrame()
for keys in data.keys():
    print(keys)
#     df = data[keys]
    complete_df = pd.concat([complete_df, data[keys]])

complete_df['Location_tags'] = (complete_df.iloc[:, 4:6] == 1).idxmax(1)    
Location_tags = complete_df['Location_tags'].str.split("-",  n=3, expand=True)
complete_df['Location_tags'] = complete_df['Location_tags'].str.split("-",  n=3, expand=True)[2]

complete_df['Outline_tags'] = (complete_df.iloc[:, 8:10] == 1).idxmax(1)
Outline_tags = complete_df['Outline_tags'].str.split("-",  n=3, expand=True)
complete_df['Outline_tags'] = complete_df['Outline_tags'].str.split("-",  n=3, expand=True)[2]

complete_df['Cysticity_tags'] = (complete_df.iloc[:, 10:13] == 1).idxmax(1)
Cysticity_tags = complete_df['Cysticity_tags'].str.split("-",  n=2, expand=True)
complete_df['Cysticity_tags'] = complete_df['Cysticity_tags'].str.replace(r'\-|\d+', '', regex=True)

complete_df['Shape_tags'] = (complete_df.iloc[:, 13:16] == 1).idxmax(1)
Shape_tags = complete_df['Shape_tags'].str.split("-",  n=3, expand=True)[2]
complete_df['Shape_tags'] = complete_df['Shape_tags'].str.split("-",  n=3, expand=True)[2]

complete_df.drop(['changed'], axis=1, inplace = True)
complete_df.drop(complete_df.iloc[:, 3:15], inplace=True, axis=1)

print(len(complete_df))
complete_df.head()

20220714_JK_d1_diff_isRASb1_corr_13_5_uM_CHIR_220018_p17_6mio_dish_2.tiff.csv
20230530_JK_BRAF_mc_8_dish_2_d20_230015.tif.csv
20230530_JK_BRAF_corr_5_dish_2_d20_230015.tif.csv
20230412_JK_diff_230011_RIT1a_corr_8_dish_1_d0.tiff.csv
20230412_JK_diff_230011_RIT1a_corr_8_dish_3_d0.tiff.csv
20230530_JK_BRAF_mc_8_dish_1_d20_230015.tif.csv
20230412_JK_diff_230011_RIT1a_corr_8_dish_2_d0.tiff.csv
20230530_JK_BRAF_corr_5_dish_1_d20_230015.tif.csv
32


,example,1-(EB) Embryonic Body,1-(NEB) non-Embryonic Body,7-Can't determine,Location_tags,Outline_tags,Cysticity_tags,Shape_tags
0,spheroid4_20220714_JK_d1_diff_isRASb1_corr_13_...,False,False,True,Edge,Smooth,Noncystic,Round
1,spheroid2_20220714_JK_d1_diff_isRASb1_corr_13_...,True,False,True,Edge,Smooth,Noncystic,Round
2,spheroid3_20220714_JK_d1_diff_isRASb1_corr_13_...,False,False,True,Edge,Smooth,Noncystic,Round
3,spheroid1_20220714_JK_d1_diff_isRASb1_corr_13_...,False,False,True,Edge,Smooth,Noncystic,Round
0,spheroid2_20230530_JK_BRAF_mc_8_dish_2_d20_230...,False,False,True,Edge,Smooth,Noncystic,Round


In [10]:
sum(complete_df["7-Can't determine"].value_counts())

32

In [5]:

print(len(complete_df))
print(sum(complete_df["7-Can't determine"]))


32
14


In [18]:
complete_df[complete_df['example'] == 'spheroid5_20220714_JK_d1_diff_isRASb1_corr_13_5_uM_CHIR_220018_p17_6mio_dish_2.tiff.png']

,example,1-(EB) Embryonic Body,1-(NEB) non-Embryonic Body,7-Can't determine,Location_tags,Outline_tags,Cysticity_tags,Shape_tags


In [17]:
names = complete_df['example']
for name in names:
    print(name)
    break

x = [complete_df.loc[complete_df['example'] == 'spheroid4_20220714_JK_d1_diff_isRASb1_corr_13_5_uM_CHIR_220018_p17_6mio_dish_2.tiff.png']['Outline_tags'].tolist(),
     complete_df.loc[complete_df['example'] == 'spheroid4_20220714_JK_d1_diff_isRASb1_corr_13_5_uM_CHIR_220018_p17_6mio_dish_2.tiff.png']['Cysticity_tags'].tolist(),
     complete_df.loc[complete_df['example'] == 'spheroid4_20220714_JK_d1_diff_isRASb1_corr_13_5_uM_CHIR_220018_p17_6mio_dish_2.tiff.png']['Shape_tags'].tolist()]

x = np.concatenate(x)
x
     


spheroid4_20220714_JK_d1_diff_isRASb1_corr_13_5_uM_CHIR_220018_p17_6mio_dish_2.tiff.png


array(['Smooth', 'Noncystic', 'Round'], dtype='<U9')

In [14]:
# complete_df.head()
# names = list(complete_df['example'])
# i = 0
# j = 0 
# for name in names:
#     i = i+1
#     nEb = complete_df[complete_df['example'] == name]["1-(NEB) non-Embryonic Body"].bool()
#     Eb = complete_df[complete_df['example'] == name]["1-(EB) Embryonic Body"].bool()
#     cantDetermine = complete_df[complete_df['example'] == name]["7-Can't determine"].bool()
# #     print('\n', name, f'\n{i}', cantDetermine, nEb, Eb)
    
#     if (cantDetermine == False) and (nEb == False) and (Eb == True):
#         j = j+1
#         print('Embryonic body - annotated - used for training')
#         print(j, 'Total training data')
#     elif (cantDetermine == False) & (nEb == True) & (Eb == False):
#         continue
#         print('Non Embryonic body - not annotated - skipped')
#     elif (cantDetermine == True) & (nEb == False) & (Eb == False):
#         continue
#         print('Less area visible - not annotated - skipped')
#     elif (cantDetermine == True) & (nEb == False) & (Eb == True):
#         continue
#         print('Less area visible - partially annotated - skipped')
    

In [15]:
complete_df[complete_df['example'] == 'spheroid4_20220714_JK_d1_diff_isRASb1_corr_13_5_uM_CHIR_220018_p17_6mio_dish_2.tiff.png' ]["1-(NEB) non-Embryonic Body"]


0    False
Name: 1-(NEB) non-Embryonic Body, dtype: bool

## checking 

In [16]:
def readData(imgPath, df = None):
    """Reads and creates a dictionary of cropped Images
    same as ----> def croppedSpheroidImg(spheroidInfo, df = None):
    """ 
    imageList = {}
    sample = {}
    print('CSV data file length:- ', len(df))
  
    for i, image_name in enumerate(sorted(os.listdir(imgPath))):
        if ((image_name.split('.')[1] == 'tif') or (image_name.split('.')[1] == 'tiff')):
            path = imgPath + image_name
            with open (path, 'rb') as f: ## rb stands for raw binary
                data = pickle.load(f)
            combinedImage = data[0]
            # mask = data[1]
            # markedImage = data[2]
            spheroidInfoDic = data[3]
            #print(i,"_total big spheroid images_", len(spheroidInfoDic))
            names = list(df['example'])
            
            ## filltering the croppedImages based on CSV file 
            for key in spheroidInfoDic:     
                croppedName = key + '.png'  # example names
                if croppedName not in  names:
#                     print(f'{croppedName} is not present for development purpose')
                    continue
                else:
                    nEb = df[df['example'] == croppedName]["1-(NEB) non-Embryonic Body"].bool()
                    Eb = df[df['example'] == croppedName]["1-(EB) Embryonic Body"].bool()
                    cantDetermine = df[df['example'] == croppedName]["7-Can't determine"].bool()
    
                    if (cantDetermine == False) and (nEb == False) and (Eb == True):
                        # print('\nEmbryonic body - annotated - used for training')
                        croppedImage = spheroidInfoDic[key][2]
                        imageList[key] = croppedImage
                        tags = [df.loc[df['example'] == croppedName]['Outline_tags'].tolist(),
                                df.loc[df['example'] == croppedName]['Cysticity_tags'].tolist(),
                                df.loc[df['example'] == croppedName]['Shape_tags'].tolist()]
                        sample = {'image': croppedImage, 'annotations': np.concatenate(tags)}
                        
                    elif (cantDetermine == False) & (nEb == True) & (Eb == False):
                        continue
                        print('Non Embryonic body - not annotated - skipped')      
                    elif (cantDetermine == True) & (nEb == False) & (Eb == False):
                        continue
                        print('Less area visible - not annotated - skipped')
                    elif (cantDetermine == True) & (nEb == False) & (Eb == True):
                        continue
                        print('Less area visible - partially annotated - skipped')   
            
        print('Total cropped spheroids after filtering', len(imageList))
            
            
    return imageList, sample

imgPath = '/kaggle/input/d/sabyasachi96/profiling-of-spheroids/bigImg_spheroidDetails/'
# imgPath = '/kaggle/input/profiling-of-spheroids/bigImg_spheroidDetails 10-44-15-277/'
imageList, sample = readData(imgPath, df = complete_df)
# print(len(imageList), imageList['spheroid1_20220714_JK_d1_diff_isRASb1_corr_13_5_uM_CHIR_220018_p17_6mio_dish_2.tiff'].shape)#, imageList.keys())
# plt.imshow(imageList['spheroid1_20220714_JK_d1_diff_isRASb1_corr_13_5_uM_CHIR_220018_p17_6mio_dish_2.tiff'])

CSV data file length:-  32
Total cropped spheroids after filtering 0
Total cropped spheroids after filtering 4
Total cropped spheroids after filtering 7
Total cropped spheroids after filtering 8
Total cropped spheroids after filtering 9
Total cropped spheroids after filtering 12
Total cropped spheroids after filtering 15
Total cropped spheroids after filtering 17


In [ ]:
def plotSanityCheckImages(imgList):
    """create a functin to check the cropped images"""
    
#     print(f'Images:{img1.shape} and mask:{img2.shape} shape\n')
    
#     figure, ax = plt.subplots(nrows=1, ncols=3, figsize=(10,10))
#     ax[0].imshow(img1)
#     ax[1].imshow(img2, cmap = 'gray')
#     ax[2].imshow(img2 * img1, cmap = 'gray')

#     ax[0].set_title("Image with the original channel")
#     ax[1].set_title("Original Mask")
#     ax[2].set_title("combined and made gray")

#     figure.tight_layout()
#     figure.show()

# plotSanityCheckImages(image[0], label[0])

## dataset Creation for cropped images

In [23]:
def combinedCSV(csvPath):
    """
    creates the complete csv file
    also preprocessing of the csv file
    """     
    with open(csvPath, 'rb') as f:
        data = pickle.load(f)
    
    combined_df = pd.DataFrame()
    for keys in data.keys():
        combined_df = pd.concat([combined_df, data[keys]])
    print(f'length of the combined df {len(combined_df)}')

    combined_df['Location_tags'] = (combined_df.iloc[:, 4:6] == 1).idxmax(1)    
    Location_tags = combined_df['Location_tags'].str.split("-",  n=3, expand=True)
    combined_df['Location_tags'] = combined_df['Location_tags'].str.split("-",  n=3, expand=True)[2]

    combined_df['Outline_tags'] = (combined_df.iloc[:, 8:10] == 1).idxmax(1)
    Outline_tags = combined_df['Outline_tags'].str.split("-",  n=3, expand=True)
    combined_df['Outline_tags'] = combined_df['Outline_tags'].str.split("-",  n=3, expand=True)[2]

    combined_df['Cysticity_tags'] = (combined_df.iloc[:, 10:13] == 1).idxmax(1)
    Cysticity_tags = combined_df['Cysticity_tags'].str.split("-",  n=2, expand=True)
    combined_df['Cysticity_tags'] = combined_df['Cysticity_tags'].str.replace(r'\-|\d+', '', regex=True)

    combined_df['Shape_tags'] = (combined_df.iloc[:, 13:16] == 1).idxmax(1)
    Shape_tags = combined_df['Shape_tags'].str.split("-",  n=3, expand=True)[2]
    combined_df['Shape_tags'] = combined_df['Shape_tags'].str.split("-",  n=3, expand=True)[2]

    combined_df.drop(['changed'], axis=1, inplace = True)
    combined_df.drop(combined_df.iloc[:, 3:15], inplace=True, axis=1)
            
    return combined_df
        
        
        
def croppedSpheroidImg(spheroidInfoDic, df = None):
    """ 
    filter the images that can be determined based on df 
    returns: imageList with their annotations 
    """
    imgList = {}
    names = list(df['example'])
    print(len(names))
    for key in spheroidInfoDic:   
        croppedName = key + '.png'  # example names
        if croppedName not in names:
#             print(f'croppedName not found--> {croppedName}')
            continue
        else:
            nEb = df[df['example'] == croppedName]["1-(NEB) non-Embryonic Body"].bool()
            Eb = df[df['example'] == croppedName]["1-(EB) Embryonic Body"].bool()
            cantDetermine = df[df['example'] == croppedName]["7-Can't determine"].bool()
    
            if (cantDetermine == False) and (nEb == False) and (Eb == True):
                croppedImage = spheroidInfoDic[key][2]
#                       imageList[key] = croppedImage
                tags = [df.loc[df['example'] == croppedName]['Outline_tags'].tolist(),
                        df.loc[df['example'] == croppedName]['Cysticity_tags'].tolist(),
                        df.loc[df['example'] == croppedName]['Shape_tags'].tolist()]
                imgList = {'image': croppedImage, 'annotations': np.concatenate(tags)}
                        
            elif (cantDetermine == False) & (nEb == True) & (Eb == False):
                continue
                print('Non Embryonic body - not annotated - skipped')      
            elif (cantDetermine == True) & (nEb == False) & (Eb == False):
                continue
                print('Less area visible - not annotated - skipped')
            elif (cantDetermine == True) & (nEb == False) & (Eb == True):
                continue
                print('Less area visible - partially annotated - skipped')   
    return imgList

In [24]:
# from torchvision import datasets
# from collections import defaultdict, deque
# import itertools

# https://discuss.pytorch.org/t/creating-custom-dataset-from-inbuilt-pytorch-datasets-along-with-data-transformations/58270
# class Cifar5000(datasets.CIFAR10):
#     def __init__(self, path, transforms, train=True):
#         super().__init__(path, train, download=True)
#         self.transforms = transforms
#         self.n_images_per_class = 5
#         self.n_classes = 10
#         self.new2old_indices = self.create_idx_mapping()

#     def create_idx_mapping(self):
#         label2idx = defaultdict(lambda: deque(maxlen=self.n_images_per_class))
#         for original_idx in range(super().__len__()):
#             _, label = super().__getitem__(original_idx)
#             label2idx[label].append(original_idx)

#         old_idxs = set(itertools.chain(*label2idx.values()))
#         new2old_indices = {}
#         for new_idx, old_idx in enumerate(old_idxs):
#             new2old_indices[new_idx] = old_idx

#         return new2old_indices

#     def __len__(self):
#         return len(self.new2old_indices)

#     def __getitem__(self, index):
#         index = self.new2old_indices[index]
#         im, label = super().__getitem__(index)
#         return self.transforms(im), label


# --------------------------------
# --------------------------------



- https://glassboxmedicine.com/2022/01/21/building-custom-image-data-sets-in-pytorch-tutorial-with-code/
- https://stackoverflow.com/questions/62434037/python-class-dataset-how-to-concatenate-images-with-their-respective-labels-in


In [25]:
class ebDataset(Dataset):
    def __init__ (self, imagePath, csvPath):
        self.imagePath = imagePath
        self.csvPath = csvPath
        self.croppedImageList = {}
        self.dataLength = self.create_idx_mapping()
        
    def create_idx_mapping(self):#, idx):
        combined_df =  combinedCSV(self.csvPath)
        print(len(combined_df), f'self.imagePath-->{self.imagePath}' )
        
        for i, image_name in enumerate(sorted(os.listdir(self.imagePath))):
            print(f'image_name--> {image_name}')
            if ((image_name.split('.')[1] == 'tif') or (image_name.split('.')[1] == 'tiff')):
                path = self.imagePath + image_name
                print(f'path--->{path}')
                with open (path, 'rb') as f: ## rb stands for raw binary
                    data = pickle.load(f)
                combinedImage = data[0] ## mask = data[1], markedImg = data[2]
                spheroidInfoDic = data[3]
                print('spheroidInfoDic', len(spheroidInfoDic), len(combined_df))
                imgList = croppedSpheroidImg(spheroidInfoDic, combined_df)
                print('imgList', imgList.keys(), len(imgList))
#                 if len(img)
                print(f"imgList['image']---> {imgList['image']}")

#                 for idx, img in enumerate(imgList):
#                     print(f'idx:{idx}----img:{img}')
#                     if idx < self.dataLength:
#                         self.croppedImageList[idx] = img
#                         print(f'croppedImageList-->{self.croppedImageList[0]})')
#                         print(f'\n{self.croppedImageList[1]}')
#                     else:
#                         break
        print('total croppedImages without filtering: ', len(self.croppedImageList))
        
        return imgList #self.croppedImageList
    
    def __len__(self):
#         length = self.
        return self.dataLength
#         return len(self.croppedImageList)
    
    def __getitem__(self, idx):
        if idx in self.croppedImageList:
            idx = self.croppedImageList[idx]
            image = idx['image']
            tags = idx['tags']
            return image, tags
        else: 
            raise KeyError(f"index{idx} not in croppedImageList")
        
           
imgPath = '/kaggle/input/d/sabyasachi96/profiling-of-spheroids/bigImg_spheroidDetails/'
csvPath = '/kaggle/input/d/sabyasachi96/profiling-of-spheroids/AllAnnotatedCSVDic.pickle'
dataset = ebDataset(imgPath, csvPath)

# for keys in dataset:
#     print(len(keys), keys.keys())
#     break
# print(len(dataset))
dataset
    

length of the combined df 32
32 self.imagePath-->/kaggle/input/d/sabyasachi96/profiling-of-spheroids/bigImg_spheroidDetails/
image_name--> divided_20220714_JK_d1_diff_isRASb1_corr_13_5_uM_CHIR_220018_p17_6mio_dish_2.tiff
path--->/kaggle/input/d/sabyasachi96/profiling-of-spheroids/bigImg_spheroidDetails/divided_20220714_JK_d1_diff_isRASb1_corr_13_5_uM_CHIR_220018_p17_6mio_dish_2.tiff
spheroidInfoDic 103 32
32
imgList dict_keys([]) 0


KeyError: 'image'

In [95]:
print(len(dataset))
for image, tags in dataset:
    print(image)
    #print(keys.shape)
#     print(len(keys[0]))
    break
# len(dataset[0])

TypeError: 'dict' object cannot be interpreted as an integer

In [ ]:
validation_split = 2
b = 1
train_set, validation_set = torch.utils.data.random_split(dataset, [len(dataset) - len(dataset)//validation_split,
                                                                    len(dataset)//validation_split])
trainLoader = DataLoader(dataset = train_set, batch_size = b, num_workers = 2, shuffle = True)
validationLoader = DataLoader(dataset = validation_set, batch_size = b, num_workers = 2, shuffle =True) 
print(f"Total {len(train_set)} examples in the training set")
print(f"Total {len(validation_set)} examples in the validation set")
print(f"Total number of training sets: {len(trainLoader)},and validation loader {len(validationLoader)}")

## creating the model 

In [4]:
class A:
    def __init__(self, item):
        self.item = item
    def __getitem__(self, index):
        return self.item[index]
a = A([1, 2, 3,4, 5])
print(f"First item: {a[0]}")
print(f"Second item: {a[1]}")
print(f"Third item: {a[2]}, {a[4]}")

First item: 1
Second item: 2
Third item: 3, 5
